### Transformer에서 GPT로 변경 차이점

1. 표준 Transformer의 Encoder는 사용하지 않고 Decoder만 사용
2. Decoder에서 Encoder의 출력과 Attention을 하는 부분인 Encoder-Decoder Multi-Head Attention 부분을 제거

# Step 0. 라이브러리 임포트

In [79]:
import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Step 1. 데이터 확인

In [2]:
import pandas as pd

data = pd.read_csv(os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ChatbotData.csv')
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


## 1.1. 데이터 확인

In [3]:
MAX_SAMPLES = len(data)
print(MAX_SAMPLES)

11823


In [4]:
data['Q'][7]

'SNS 맞팔 왜 안하지ㅠㅠ'

In [5]:
data['A'][7]

'잘 모르고 있을 수도 있어요.'

1. 문장의 시작에는 Start token을 넣어주고, 문장의 끝에는 Extract toekn을 넣어 주었다.
여기서 extract token은 EOS 기능 뿐만 아니라 이후 downwrad task의 query 벡터로 활용.Delim(delimiter) token이 활용되는데, 해당 토큰은 서로 다른 두 문장을 이어주는 역할.
2. Multiple Choice
문맥 문서 z, 질문 q, 가능한 답변 a_k라 하면 [z; q; delim token; a_k]로 연결하고 input 개수는 답변의 갯수만큼 생성
GPT의 이러한 input 구조의 장점은 transformer 구조를 별도의 학습 없이 여러 task에서 활용가능하다는 것이다.
사용자는 down-stream task를 위한 마지막 linear layer만 별도로 학습 시켜(fine-tunning) 원하는 task에 활용

# Step 2. 데이터 전처리하기

In [6]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # ("a-z가-힣",".","?","!",",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-z가-힣?.!,]+", " ", sentence) #한글 전처리
    sentence = sentence.strip()
    return sentence

In [7]:
# 영어 대문자 -> 소문자
preprocess_sentence(data['Q'][7])
# 숫자 제외
preprocess_sentence(data['Q'][3])

'박 일 정도 놀러가고 싶다'

Q, A, label -> Q + delim token + A

In [8]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수
def load_conversations():
    text = []

    for i in range(len(data)):
      # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용.
        text.append(preprocess_sentence(data['Q'][i]+","+data['A'][i]))
        
       
        if len(text) >= MAX_SAMPLES:
            return text
        
    return text

In [9]:
text = load_conversations()

In [10]:
print('전체 샘플 수 :', len(text))

전체 샘플 수 : 11823


In [11]:
print('전처리 후의 샘플: {}'.format(text[3]))

전처리 후의 샘플: 박 일 정도 놀러가고 싶다 , 여행은 언제나 좋죠 .


In [12]:
print('전처리 후의 샘플: {}'.format(text[:7]))

전처리 후의 샘플: ['시 땡 ! , 하루가 또 가네요 .', '지망 학교 떨어졌어 , 위로해 드립니다 .', '박 일 놀러가고 싶다 , 여행은 언제나 좋죠 .', '박 일 정도 놀러가고 싶다 , 여행은 언제나 좋죠 .', 'ppl 심하네 , 눈살이 찌푸려지죠 .', 'sd카드 망가졌어 , 다시 새로 사는 게 마음 편해요 .', 'sd카드 안돼 , 다시 새로 사는 게 마음 편해요 .']


In [14]:
max_len = max([len(line) for line in text])
print('문장의 최대 길이 :',max_len)


문장의 최대 길이 : 97


# Step 3. SubwordTextEncoder 사용하기

## 3.1 단어장 만들기 

In [23]:
#각 단어에 고유한 정수 인덱스를 부여하기 위해 생성
import tensorflow_datasets as tfds

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(text, target_vocab_size=2**13)

In [24]:
print(tokenizer.subwords[:100])

[' .', ' , ', ' . , ', ' ? , ', '거예요', '수_', '게_', '너무_', '더_', '거_', '는_', '좋아하는_', ' . ', '이_', '을_', '잘_', '도_', '고_', '요', '것_', '많이_', '안_', '좋은_', '같아요', '한_', '좀_', '있어요', '싶어', '나_', '가_', '있을_', '에_', '지_', '해보세요', '은_', '사람_', '할_', '같아', '해', '네', '면_', '건_', '사람이_', '를_', '마세요', '다_', '하고_', '지', '하는_', '보세요', '죠', '어', '서_', '내가_', '다', '내_', '이제_', '마음이_', '의_', '나', ' !', '다른_', '썸_', '만_', '그_', '어떻게_', '있는_', '왜_', '싶다', '세요', '다시_', '시간이_', '수도_', '없어', '것도_', '또_', '좋을_', '오늘_', '정말_', '가', '이', '같이_', '네요', '될_', '해요', '자꾸_', '일_', '걸_', '있어', '하세요', '없어요', '제가_', '길_', '바랄게요', '로_', '까', '돼요', '하면_', '봐요', '할까']


In [15]:
print('Tokenized sample : {}'.format(tokenizer.encode(text[7])))

Tokenized sample : [1950, 646, 1127, 7917, 68, 5095, 2, 16, 1894, 31, 73, 27, 1]


In [25]:
sample_string = text[7]
print('text[7] 원본 : {}'.format(sample_string))

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))


text[7] 원본 : sns 맞팔 왜 안하지 , 잘 모르고 있을 수도 있어요 .
정수 인코딩 후의 문장 : [1950, 646, 1127, 7917, 68, 5095, 2, 16, 1894, 31, 73, 27, 1]
기존 문장 : sns 맞팔 왜 안하지 , 잘 모르고 있을 수도 있어요 .


In [26]:
print('단어 집합의 크기(Vocab size) :', tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8141


In [27]:
for ts in tokenized_string:
    print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

1950 ----> sns 
646 ----> 맞
1127 ----> 팔
7917 ---->  
68 ----> 왜 
5095 ----> 안하지
2 ---->  , 
16 ----> 잘 
1894 ----> 모르고 
31 ----> 있을 
73 ----> 수도 
27 ----> 있어요
1 ---->  .


In [28]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

#시작 토큰과 종료 토큰에 부여된 정수를 출력
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print('VOCAB_SIZE :' , VOCAB_SIZE)

START_TOKEN의 번호 : [8141]
END_TOKEN의 번호 : [8142]
VOCAB_SIZE : 8143


In [30]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 92
# MAX_LENGTH = 30
# MAX_LENGTH = 10

print(MAX_LENGTH)

# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(text):
    tokenized = []

    for t in text:
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence = START_TOKEN + tokenizer.encode(t) + END_TOKEN

        # 최대 길이 이하인 경우에만 데이터셋으로 허용
        if len(sentence) <= MAX_LENGTH :
            tokenized.append(sentence)

    # 최대 길이로 모든 데이터셋을 패딩
    tokenized = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized, maxlen=MAX_LENGTH, padding='post')
  
    return tokenized

92


In [31]:
text = tokenize_and_filter(text)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 샘플 개수: {}'.format(len(text)))

단어장의 크기 : 8143
필터링 후의 샘플 개수: 11823


1. 인코더의 셀프 어텐션 : Query = Key = Value
2. 디코더의 마스크드 셀프 어텐션 : Query = Key = Value
3. 디코더의 인코더-디코더 어텐션 : Query : 디코더 벡터 / Key = Value : 인코더 벡터

## 3.2. 교사 강요(Teacher Forcing) 사용하기

교사 강요란?
보통 RNN은 (n-1) 스텝에서의 출력 값을 n 스텝의 입력값으로 사용.
교사 강요는 (n-1) 스텝의 예측 값을 n 스텝의 입력값으로 사용하는 것이 아니라 (n-1) 스텝의 실제값을 n 스텝의 입력값으로 넣어주는 방식이다

In [32]:
BATCH_SIZE = 128
# BUFFER_SIZE = 20000
BUFFER_SIZE = MAX_LENGTH


# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
# 교사강요를 위해 
# answers[:, :-1] -> 디코더의 입력값 , answers[:, 1:] -> 디코더의 레이블 사용
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': text,
        'dec_inputs': text[:, :-1]
    },
    {
        'outputs': text[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [33]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [89]:
dataset

<PrefetchDataset shapes: ({inputs: (None, 92), dec_inputs: (None, 91)}, {outputs: (None, 91)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>

# Step 4. 모델 구성하기

0. GPT-1은 Byte pair encoding을 사용한다
1. 12개의 decoder-only transformer layer 활용
2. 12개의 multihead
3. 인코딩 벡터의 차원 768
4. ReLu와 비슷한 생김새를 가진 GELU 활성화 함수 사용

## 4.1. Text & Position embed

In [90]:
text

array([[8141, 4191, 3049, ...,    0,    0,    0],
       [8141,   48,  920, ...,    0,    0,    0],
       [8141, 2997,   87, ...,    0,    0,    0],
       ...,
       [8141, 8122, 8042, ...,    0,    0,    0],
       [8141,  136,  179, ...,    0,    0,    0],
       [8141, 1952,  886, ...,    0,    0,    0]], dtype=int32)

## 4.2. 스케일드 닷 프로덕트 어텐션

In [64]:
def scaled_dot_product_attention(query, key, value, mask):
    # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9) 

    # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    return output

## 4.3. 멀티 헤드 어텐션

In [47]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

## 4.4. 패딩 마스킹 

In [65]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

## 4.5. 룩 어헤드 마스킹

In [67]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

## 4.7. 디코더

In [68]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

    # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
#     attention2 = MultiHeadAttention(
#       d_model, num_heads, name="attention_2")(inputs={
#           'query': attention1,
#           'key': enc_outputs,
#           'value': enc_outputs,
#           'mask': padding_mask
#       })

    # 마스크드 멀티 헤드 어텐션의 결과는
    # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    # 세 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [70]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

    # 패딩 마스크
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

## 4.8. 트랜스포머

In [72]:
#앞서 사용한 인코더 층 함수와 디코더 층 함수를 사용하여 트랜스포머 함수를 정의
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
    # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

    # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    # 디코더
    dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

## 4.9. 모델생성

In [73]:
tf.keras.backend.clear_session()

# 하이퍼파라미터 
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수
D_MODEL = 512 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수
UNITS = 2048 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

'''
vocab_size = 20000  # Only consider the top 20k words
maxlen = 80  # Max sequence size
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer

'''

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

ModuleNotFoundError: No module named 'keras.engine.base_layer_v1'

## 4.10. Loss function

In [ ]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)
print("슝=3")

## 4.11. Learning rate

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
sample_learning_rate = CustomSchedule(d_model=128)

plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

## 4.12. 모델 컴파일

In [ ]:
learning_rate = CustomSchedule(D_MODEL)


optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

## 4.13 모델 훈련하기 

In [ ]:
EPOCHS = 50
# EPOCHS = 100
model.fit(dataset, epochs=EPOCHS, verbose=1)

# Step 5. 모델 평가하기

In [ ]:
#예측(inference)으로 챗봇 테스트하기
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
        # 이 output_sequence는 다시 디코더의 입력이 됩니다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

print("슝=3")

In [ ]:
#임의의 입력 문장에 대해서 decoder_inference() 함수를 호출하여 챗봇의 대답을 얻는 sentence_generation() 함수를 만듭니다.
def sentence_generation(sentence):
    # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

print("슝=3")

In [ ]:
#임의의 문장으로부터 챗봇의 대답을 얻어봅시다.
sentence_generation('오늘 날씨는 어때?')

In [ ]:
#임의의 문장으로부터 챗봇의 대답을 얻어봅시다.
sentence_generation('머리가 복잡해')

In [ ]:
#임의의 문장으로부터 챗봇의 대답을 얻어봅시다.
sentence_generation('기분이 안 좋아')

In [ ]:
#임의의 문장으로부터 챗봇의 대답을 얻어봅시다.
sentence_generation('저녁 뭐 먹을지 고민돼')

# 회고

In [ ]:
'''
빠진 수업내용 보고 논문 읽어보자
1. 노드 5~8
2. 어텐션 -> 트랜스포머 -> gpt
'''